4월 데이터 결합 및 지금까지 코드 정리

# 데이터 로드 및 라이브러리 임포트

In [ ]:
# 라이브러리 임포트
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.spatial import distance
from math import log2
from math import sqrt
from sklearn.cluster import AgglomerativeClustering
import time
import warnings
warnings.filterwarnings(action = 'ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data_1=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/201901-202003.csv')
data_2=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/202004.csv')

In [ ]:
data_1.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [ ]:
data_2.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,3,779600,4
1,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,2,3,14,2427700,15
2,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,1,4,18,2184600,18
3,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,2,4,21,3653100,23
4,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,60s,1,5,15,2765500,18


In [ ]:
data = pd.concat([data_1,data_2])

In [ ]:
data.isna().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      92371
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      155234
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
dtype: int64

두 데이터 모두 결측치가 있기 때문에 처리해주어야한다.
- 파일: Jeju_DataPreprocess_02th


In [ ]:
data.loc[data['CARD_SIDO_NM']=='세종','CARD_CCG_NM']='세종'
data.loc[data['HOM_SIDO_NM']=='세종','HOM_CCG_NM']='세종'

In [ ]:
data.isna().sum()

REG_YYMM        0
CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
dtype: int64

In [ ]:
# 테스트 파일제작 
df_form = data.groupby(list(data.drop(['AMT','CSTMR_CNT','CNT'],axis=1).columns[1:]))['AMT'].sum().reset_index().drop('AMT',axis=1)
df_form['REG_YYMM'] = 202007
test_df= df_form.copy()

test_df[list(data.drop(['AMT','CSTMR_CNT','CNT'],axis=1).columns)]
test_df.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,REG_YYMM
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,202007
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1,202007
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,202007
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,202007
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,202007


# 파생변수 제작
- Jensen Shannon divergence 적용
- 합체 군집 분석 적용
- 우리나라 코로나 첫 확진자 발생일 2020년 1월 8일
- 전체기간 Jensen Shannon,19년 Jensen Shannon, 19년 12월- 20년 4월 Jensen Shannon 식으로 총 3가지 접근의 군집을 만들 필요가 있을 듯

In [ ]:
def Jensenshannon_similarity(df,input_list):
  '''
  [함수 사용법]
    REG_YYMM을 input_list 인덱싱 0번 자리(맨앞)에 둘 것
    그 뒤로 기준을 잡고 싶은 변수들을 차례로 넣어 주면 됩니다.
    from scipy.spatial import distance 꼭 할것!
  '''
  
  make_amt= df.groupby(input_list)['AMT'].sum().reset_index()
  
  '''pivot 만들기'''
  amt_sum = make_amt.groupby(input_list[1:])['AMT'].sum().reset_index(name='AMT_sum')
  make_amt = make_amt.merge(amt_sum,on=input_list[1:],how='left')
  make_amt['AMT_prob'] = make_amt['AMT']/make_amt['AMT_sum']

  # pivot_table을 활용하여 long to wide 작업 실시
  amt_pivot= pd.pivot_table(make_amt,
                            index=input_list[1:],
                            columns='REG_YYMM',
                            values='AMT_prob').reset_index()
  amt_pivot.fillna(0,inplace=True)

  #피벗된 연도별 amt를 사용해 고객간의 유사도를 구해보자.
  amt_array=np.array(amt_pivot.drop(input_list[1:],axis=1))

  js_similarity =[]
  for row_r in range(amt_array.shape[0]):       
    dt_list=[]
    for row_c in range(amt_array.shape[0]):
        #dt= js_divergence(amt_pivot.iloc[row_r,3:].values,amt_pivot.iloc[row_c,3:].values)
        '''
        js_divergence(만든 함수)로 하면 0값을 계산을 못함. 
        sqrt(js_divergence)인 distance.jensenshannon 함수 사용
        '''
        dt = distance.jensenshannon(amt_array[row_r,:],amt_array[row_c,:],2.0)
        dt_list.append(dt)
    js_similarity.append(dt_list)

  js_frame =pd.DataFrame(np.array(js_similarity))
  js_similarity = pd.concat([amt_pivot,js_frame],axis=1)

  return js_similarity

In [ ]:
def make_clustering(gp_df,input_list,col_name):
    df = Jensenshannon_similarity(gp_df,input_list)
    df_s = df.iloc[:,-df.shape[0]:] # 클러스터링용 dataframe 생성
    clustering = AgglomerativeClustering(n_clusters=int(df_s.shape[0]/2),
                                     affinity='precomputed',
                                     linkage='average')
    clustering.fit(df_s)
    
    dfd = df.iloc[:,:-df.shape[0]]
    dfd['cluster'] = clustering.labels_
    dfd.rename(columns={'cluster':col_name},inplace=True)
    
    print(dfd.columns[-1],'success!')

    input_list.append(dfd.columns[-1])
    return dfd[input_list[1:]]

In [ ]:
# 업종 clustering total

STD_total = make_clustering(data,['REG_YYMM','STD_CLSS_NM'],'STD_total')
SIDO_STD_total = make_clustering(data,['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM'],'SIDO_STD_total')
SIDO_CCG_STD_total = make_clustering(data,['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM'],'SIDO_CCG_STD_total')

# 소비자 clustering total

ASF_total = make_clustering(data,['REG_YYMM','AGE','SEX_CTGO_CD','FLC'],'ASF_total')
SIDO_ASF_total = make_clustering(data,['REG_YYMM', 'HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_ASF_total')
SIDO_CCG_ASF_total = make_clustering(data,['REG_YYMM', 'HOM_SIDO_NM', 'HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_CCG_ASF_total')
SIDO_ASF_2_total = make_clustering(data,['REG_YYMM', 'CARD_SIDO_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_ASF_2_total')
SIDO_CCG_ASF_2_total = make_clustering(data,['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_CCG_ASF_2_total')

STD_total success!
SIDO_STD_total success!
SIDO_CCG_STD_total success!
ASF_total success!
SIDO_ASF_total success!
SIDO_CCG_ASF_total success!
SIDO_ASF_2_total success!
SIDO_CCG_ASF_2_total success!


In [ ]:
# 업종 clustering 2019

STD_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM','STD_CLSS_NM'],'STD_2019')
SIDO_STD_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM'],'SIDO_STD_2019')
SIDO_CCG_STD_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM'],'SIDO_CCG_STD_2019')

# 소비자 clustering 2019

ASF_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM','AGE','SEX_CTGO_CD','FLC'],'ASF_2019')
SIDO_ASF_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM', 'HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_ASF_2019')
SIDO_CCG_ASF_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM', 'HOM_SIDO_NM', 'HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_CCG_ASF_2019')
SIDO_ASF_2_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM', 'CARD_SIDO_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_ASF_2_2019')
SIDO_CCG_ASF_2_2019 = make_clustering(data[data['REG_YYMM']<202000],['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_CCG_ASF_2_2019')

STD_2019 success!
SIDO_STD_2019 success!
SIDO_CCG_STD_2019 success!
ASF_2019 success!
SIDO_ASF_2019 success!
SIDO_CCG_ASF_2019 success!
SIDO_ASF_2_2019 success!
SIDO_CCG_ASF_2_2019 success!


In [ ]:
# 업종 clustering 2020

STD_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM','STD_CLSS_NM'],'STD_2020')
SIDO_STD_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM'],'SIDO_STD_2020')
SIDO_CCG_STD_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM'],'SIDO_CCG_STD_2020')

# 소비자 clustering 2020

ASF_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM','AGE','SEX_CTGO_CD','FLC'],'ASF_2020')
SIDO_ASF_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM', 'HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_ASF_2020')
SIDO_CCG_ASF_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM', 'HOM_SIDO_NM', 'HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_CCG_ASF_2020')
SIDO_ASF_2_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM', 'CARD_SIDO_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_ASF_2_2020')
SIDO_CCG_ASF_2_2020 = make_clustering(data[data['REG_YYMM']>=201912],['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM','AGE','SEX_CTGO_CD','FLC'],'SIDO_CCG_ASF_2_2020')

STD_2020 success!
SIDO_STD_2020 success!
SIDO_CCG_STD_2020 success!
ASF_2020 success!
SIDO_ASF_2020 success!
SIDO_CCG_ASF_2020 success!
SIDO_ASF_2_2020 success!
SIDO_CCG_ASF_2_2020 success!


In [ ]:
# 군집화 결과 삽입
for md in [STD_total,SIDO_STD_total,SIDO_CCG_STD_total,ASF_total,SIDO_ASF_total,SIDO_CCG_ASF_total,SIDO_ASF_2_total,SIDO_CCG_ASF_2_total,
           STD_2019,SIDO_STD_2019,SIDO_CCG_STD_2019,ASF_2019,SIDO_ASF_2019,SIDO_CCG_ASF_2019,SIDO_ASF_2_2019,SIDO_CCG_ASF_2_2019,
           STD_2020,SIDO_STD_2020,SIDO_CCG_STD_2020,ASF_2020,SIDO_ASF_2020,SIDO_CCG_ASF_2020,SIDO_ASF_2_2020,SIDO_CCG_ASF_2_2020]:
  data = data.merge(md,on=list(md.columns[:-1]),how='left')
  test_df = test_df.merge(md,on=list(md.columns[:-1]),how='left')

In [ ]:
data.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/mid_train_df.csv',encoding='utf-8-sig',index=False)
test_df.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/mid_test_df.csv',encoding='utf-8-sig',index=False)

In [ ]:
data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,STD_total,SIDO_STD_total,SIDO_CCG_STD_total,ASF_total,SIDO_ASF_total,SIDO_CCG_ASF_total,SIDO_ASF_2_total,SIDO_CCG_ASF_2_total,STD_2019,SIDO_STD_2019,SIDO_CCG_STD_2019,ASF_2019,SIDO_ASF_2019,SIDO_CCG_ASF_2019,SIDO_ASF_2_2019,SIDO_CCG_ASF_2_2019,STD_2020,SIDO_STD_2020,SIDO_CCG_STD_2020,ASF_2020,SIDO_ASF_2020,SIDO_CCG_ASF_2020,SIDO_ASF_2_2020,SIDO_CCG_ASF_2_2020
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,16,278,2484,11,77,248,201,1746,10,35.0,2468.0,11,195,1150.0,223.0,2030.0,13,247.0,3222.0,6,99.0,7.0,115.0,762.0
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,16,278,2484,0,5,125,47,173,10,35.0,2468.0,2,5,70.0,4.0,812.0,13,247.0,3222.0,3,20.0,74.0,160.0,1610.0
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,16,278,2484,2,37,96,179,173,10,35.0,2468.0,0,17,70.0,4.0,812.0,13,247.0,3222.0,0,219.0,84.0,125.0,77.0
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,16,278,2484,0,5,125,41,295,10,35.0,2468.0,2,12,142.0,96.0,133.0,13,247.0,3222.0,3,20.0,167.0,114.0,3168.0
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,16,278,2484,0,0,44,54,158,10,35.0,2468.0,2,8,0.0,38.0,83.0,13,247.0,3222.0,3,12.0,74.0,1.0,597.0


In [ ]:
test_df.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,REG_YYMM,STD_total,SIDO_STD_total,SIDO_CCG_STD_total,ASF_total,SIDO_ASF_total,SIDO_CCG_ASF_total,SIDO_ASF_2_total,SIDO_CCG_ASF_2_total,STD_2019,SIDO_STD_2019,SIDO_CCG_STD_2019,ASF_2019,SIDO_ASF_2019,SIDO_CCG_ASF_2019,SIDO_ASF_2_2019,SIDO_CCG_ASF_2_2019,STD_2020,SIDO_STD_2020,SIDO_CCG_STD_2020,ASF_2020,SIDO_ASF_2020,SIDO_CCG_ASF_2020,SIDO_ASF_2_2020,SIDO_CCG_ASF_2_2020
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,202007,16,278,2484,11,77,248,201,1746,10,35.0,2468.0,11,195,1150.0,223.0,2030.0,13,247.0,3222.0,6,99.0,7.0,115.0,762.0
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1,202007,16,278,2484,9,9,155,181,3091,10,35.0,2468.0,8,39,3086.0,129.0,2755.0,13,247.0,3222.0,2,41.0,1310.0,74.0,359.0
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,202007,16,278,2484,0,5,125,47,173,10,35.0,2468.0,2,5,70.0,4.0,812.0,13,247.0,3222.0,3,20.0,74.0,160.0,1610.0
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,202007,16,278,2484,2,37,96,179,173,10,35.0,2468.0,0,17,70.0,4.0,812.0,13,247.0,3222.0,0,219.0,84.0,125.0,77.0
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,202007,16,278,2484,0,5,125,41,295,10,35.0,2468.0,2,12,142.0,96.0,133.0,13,247.0,3222.0,3,20.0,167.0,114.0,3168.0
